In [1]:
import pandas as pd
# 1. Load S&P 500 constituents (Symbol, GICS Sector, etc.)
sp500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
# 2. Define sectors/themes for brown vs green
brown_sectors = ["Energy"]          # e.g. fossil-fuel producers
green_sectors = ["Utilities"]      # e.g. power producers (often renewables)
green_sectors += ["Information Technology"]  # tech as a proxy for innovation
# 3. Filter tickers by these sectors
brown = sp500[sp500["GICS Sector"].isin(brown_sectors)]["Symbol"].tolist()
green = sp500[sp500["GICS Sector"].isin(green_sectors)]["Symbol"].tolist()
candidates = brown + green
print(f"Selected {len(candidates)} tickers: {len(brown)} brown, {len(green)} green")


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Selected 122 tickers: 23 brown, 99 green


In [8]:
# ## 4. Fetch ESG Scores via yfinance (revised)
import numpy as np
import yfinance as yf
esg_scores = {}

def fetch_esg_score(ticker):
    info = yf.Ticker(ticker).sustainability
    if info is None:
        return np.nan

    # normalize index to lowercase for matching
    idx = info.index.str.lower()

    # 1. try totalEsg
    if "totalesg" in idx:
        return float(info.iloc[idx.get_loc("totalesg")][0])

    # 2. else average the three if present
    subs = ["environmentscore", "socialscore", "governancescore"]
    if all(s in idx for s in subs):
        vals = [float(info.iloc[idx.get_loc(s)][0]) for s in subs]
        return float(np.mean(vals))

    return np.nan

esg_scores = {t: fetch_esg_score(t) for t in candidates}

# build and inspect the DataFrame
esg_df = pd.Series(esg_scores, name="ESG").to_frame()
print(esg_df.describe().round(1))

# show bottom & top ten by ESG
print("Lowest ESG scores:\n", esg_df.sort_values("ESG").head(10))
print("Highest ESG scores:\n", esg_df.sort_values("ESG", ascending=False).head(10))



/var/folders/z8/lprld2kj00j12npqpk_410yr0000gn/T/ipykernel_21843/2394479798.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return float(info.iloc[idx.get_loc("totalesg")][0])
/var/folders/z8/lprld2kj00j12npqpk_410yr0000gn/T/ipykernel_21843/2394479798.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return float(info.iloc[idx.get_loc("totalesg")][0])
/var/folders/z8/lprld2kj00j12npqpk_410yr0000gn/T/ipykernel_21843/2394479798.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior)

         ESG
count  122.0
mean    21.6
std      8.0
min      7.5
25%     16.0
50%     19.2
75%     26.8
max     42.9
Lowest ESG scores:
         ESG
CDW    7.49
JBL    8.98
KEYS   9.06
TRMB   9.55
ZBRA   9.94
HPQ   10.59
ACN   11.09
AMAT  11.43
STX   11.55
HPE   11.67
Highest ESG scores:
         ESG
XOM   42.89
APA   38.35
EOG   38.00
CTRA  37.53
HES   37.37
FANG  37.31
CVX   36.96
EXE   36.35
COP   35.94
TRGP  35.80


/var/folders/z8/lprld2kj00j12npqpk_410yr0000gn/T/ipykernel_21843/2394479798.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return float(info.iloc[idx.get_loc("totalesg")][0])
/var/folders/z8/lprld2kj00j12npqpk_410yr0000gn/T/ipykernel_21843/2394479798.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return float(info.iloc[idx.get_loc("totalesg")][0])


In [14]:
# 3) Compute within-sector quartiles
brown_esg = esg_df.loc[brown, "ESG"].dropna()
green_esg = esg_df.loc[green, "ESG"].dropna()

# bottom 25% of Energy
q1_brown = brown_esg.quantile(0.25)
brown_low  = brown_esg[brown_esg <= q1_brown].index.tolist()

# top 25% of Utilities+IT
q3_green = green_esg.quantile(0.75)
green_high = green_esg[green_esg >= q3_green].index.tolist()

print(f"Brown_low (bottom 25% Energy, n={len(brown_low)}): {brown_low}")
print(f"Green_high (top 25% Utilities+IT, n={len(green_high)}): {green_high}")


Brown_low (bottom 25% Energy, n=6): ['BKR', 'HAL', 'KMI', 'SLB', 'TPL', 'WMB']
Green_high (top 25% Utilities+IT, n=25): ['AES', 'AEE', 'AEP', 'ATO', 'CNP', 'CMS', 'CEG', 'D', 'DTE', 'DUK', 'ETR', 'EPAM', 'EVRG', 'FE', 'NEE', 'NRG', 'PCG', 'PNW', 'PPL', 'PEG', 'SWKS', 'SO', 'TDY', 'VST', 'XEL']


In [17]:
import pandas as pd
import yfinance as yf

import os


PROCESSED_DIR = "../data/processed"
os.makedirs(PROCESSED_DIR, exist_ok=True)
start_date = "2015-01-01"
end_date   = "2025-01-01"
# 4. Download historical price data
selected = brown_low + green_high
df_adj = yf.download(
    selected,
    start=start_date,
    end=end_date,
    progress=False
)["Close"]
# 5. Drop tickers with no data
df_adj.dropna(axis=1, how="all", inplace=True)

# 6. Align on common dates (drop any rows with missing prices)
aligned = df_adj.dropna(axis=0, how="any")
print(f"Aligned: {aligned.shape[0]} days × {aligned.shape[1]} tickers")

out_path = os.path.join(PROCESSED_DIR, "aligned_prices2.csv")
aligned.to_csv(out_path)
print("Saved aligned prices →", out_path)

/var/folders/z8/lprld2kj00j12npqpk_410yr0000gn/T/ipykernel_21843/637146149.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_adj = yf.download(


Aligned: 742 days × 31 tickers
Saved aligned prices → ../data/processed/aligned_prices2.csv


In [18]:
import pandas_market_calendars as mcal

# 7. Identify missing trading days
aligned_naive = aligned.copy()
aligned_naive.index = aligned_naive.index.tz_localize(None)  # remove timezone

start_date = aligned_naive.index.min().date()
end_date   = aligned_naive.index.max().date()

nyse = mcal.get_calendar("NYSE")
schedule = nyse.schedule(start_date=start_date, end_date=end_date).index.tz_localize(None)
missing_days = schedule.difference(aligned_naive.index)
print(f"Missing trading days (should be market holidays): {len(missing_days)} dates\n{missing_days}")


Missing trading days (should be market holidays): 0 dates
DatetimeIndex([], dtype='datetime64[ns]', freq=None)


In [23]:
# 8. Compute daily returns and correlation
returns = aligned.pct_change().dropna()
corr = returns.corr()

# 9. Identify top correlated pairs (for example, correlation > 0.8)
pairs = []
tickers = returns.columns.tolist()
for i in range(len(tickers)):
    for j in range(i+1, len(tickers)):
        r = corr.iloc[i,j]
        pairs.append((tickers[i], tickers[j], r))
pairs_sorted = sorted(pairs, key=lambda x: -x[2])
top20 = pairs_sorted[:20]
print("Top 20 correlated pairs:", top20)

bg_pairs = [
    (a, b, round(rho,3))
    for a,b,rho in top20
    if (a in brown_low and b in green_high) or (b in brown_low and a in green_high)
]
print(f"{len(bg_pairs)} of the Top 20 pairs are brown↔green:")
for a,b,rho in bg_pairs:
    print(f"  • {a} – {b}: ρ = {rho}")


Top 20 correlated pairs: [('CMS', 'DTE', 0.8851117753399977), ('AEE', 'CMS', 0.8634214032730332), ('HAL', 'SLB', 0.8546640533378912), ('CMS', 'DUK', 0.84978508581229), ('DUK', 'SO', 0.8490992486859741), ('KMI', 'WMB', 0.8474650889725196), ('AEP', 'CMS', 0.8448450014838709), ('DTE', 'DUK', 0.8418937644620494), ('AEP', 'DUK', 0.8331954483407175), ('AEE', 'DTE', 0.8288178365868913), ('DTE', 'EVRG', 0.8280157725205463), ('AEP', 'DTE', 0.8248220336935643), ('AEP', 'EVRG', 0.8241645109297682), ('CMS', 'EVRG', 0.8216455206806815), ('CMS', 'SO', 0.8195892313056087), ('AEE', 'EVRG', 0.8192025477862901), ('AEP', 'SO', 0.8177700686173645), ('DUK', 'EVRG', 0.8127132005909943), ('AEE', 'DUK', 0.8103070770227307), ('DTE', 'SO', 0.8058430302472777)]
0 of the Top 20 pairs are brown↔green:


In [25]:
# 1) Build the full correlation list
pairs = []
tickers = returns.columns.tolist()
for i in range(len(tickers)):
    for j in range(i+1, len(tickers)):
        ρ = corr.iloc[i, j]
        pairs.append((tickers[i], tickers[j], ρ))

# 2) Filter only brown_low ↔ green_high pairs
cross_pairs = [
    (a, b, rho)
    for a, b, rho in pairs
    if (a in brown_low and b in green_high)
    or (b in brown_low and a in green_high)
]

# 3) Sort by descending correlation, pick, say, the top 10 cross-group candidates
top_cross = sorted(cross_pairs, key=lambda x: -x[2])[:10]
print("Top brown↔green correlation candidates:")
for a, b, rho in top_cross:
    print(f"  • {a}–{b}: ρ = {rho:.3f}")

# 4) Save them for cointegration testing
import json
out_path_corr = os.path.join(PROCESSED_DIR, "brown_green_corr_pairs.json")

with open(out_path_corr, "w") as f:
    json.dump([{"A":a,"B":b,"corr":rho} for a,b,rho in top_cross], f, indent=2)


Top brown↔green correlation candidates:
  • ATO–KMI: ρ = 0.507
  • KMI–PPL: ρ = 0.493
  • CNP–KMI: ρ = 0.473
  • ATO–WMB: ρ = 0.473
  • DTE–KMI: ρ = 0.458
  • AEE–KMI: ρ = 0.455
  • KMI–PNW: ρ = 0.452
  • CNP–WMB: ρ = 0.451
  • FE–KMI: ρ = 0.447
  • EVRG–KMI: ρ = 0.444
